# Importing Libraries

In [ ]:
pip install --upgrade tensorflow keras


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

In [ ]:
import cv2
import numpy as np
from skimage.feature import greycomatrix, greycoprops
from tensorflow.keras.models import load_model
import pickle

# Load the trained model and scaler
model = load_model('model.h5')
scaler = pickle.load(open('scaler1.pkl', 'rb'))

def extract_features(image_path):
    # Load and preprocess the image
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Extract GLCM features
    glcm = greycomatrix(gray_image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)

    features = {}
    features['Contrast'] = greycoprops(glcm, 'contrast')[0, 0]
    features['Correlation'] = greycoprops(glcm, 'correlation')[0, 0]
    features['Energy'] = greycoprops(glcm, 'energy')[0, 0]
    features['Homogeneity'] = greycoprops(glcm, 'homogeneity')[0, 0]

    # Calculate other features
    features['Mean'] = np.mean(gray_image)
    features['Standard Deviation'] = np.std(gray_image)
    features['Entropy'] = -np.sum(glcm * np.log2(glcm + (glcm==0)))
    features['Variance'] = np.var(gray_image)

    # Calculate skewness and kurtosis
    mean = np.mean(gray_image)
    features['Skewness'] = np.mean(((gray_image - mean) / np.std(gray_image)) ** 3)
    features['Kurtosis'] = np.mean(((gray_image - mean) / np.std(gray_image)) ** 4) - 3

    return features

def predict_bruise(image_path):
    # Extract features
    features = extract_features(image_path)

    # Prepare feature vector
    feature_vector = np.array([[
        features['Contrast'], features['Correlation'], features['Energy'],
        features['Homogeneity'], features['Mean'], features['Standard Deviation'],
        features['Entropy'], features['Variance'], features['Kurtosis'],
        features['Skewness']
    ]])

    # Scale features
    scaled_features = scaler.transform(feature_vector)

    # Make prediction
    prediction = model.predict(scaled_features)

    return prediction[0]

# Test the model
image_path = '/content/20.jpg'  # Replace with your image path
prediction = predict_bruise(image_path)

print(f"Predicted probabilities: Non-Bruised={prediction[0]:.4f}, Bruised={prediction[1]:.4f}")

threshold = 0.5
if prediction[1] >= threshold:
    print("Prediction: Bruised")
else:
    print("Prediction: Non-Bruised")

/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimag

1/1 [==============================] - 0s 416ms/step
Predicted probabilities: Non-Bruised=0.7915, Bruised=0.2085
Prediction: Non-Bruised


In [ ]:
pip install tensorflow==2.16.1
